In [52]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler # Look at RF for package
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score
import sklearn.metrics as mt
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from scipy import stats
from scipy import sparse

import random
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

import numpy as np
np.random.seed(20170301)

# Zipcode train, test, predict data

In [22]:
# get 2013, 2014, and 2015 data for zipcode

init_zip_2013 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_zipcode_2013.csv')
init_zip_2014 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_zipcode_2014.csv')
init_zip_2015 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_zipcode_2015.csv')

### ^^^ DJC NOTE: I've changed the name of the initialized dataframes here and created copies below keep from over-writing them when you drop the columns, like "geometry" and later "zipcode" from the features and target matrices. Now you can easily merge your predicted gas leaks per zip back to this initial df that has zip code number and geometry for mapping, etc. (and the index is unchanged, so rows should be in exact same order).

In [23]:
# create copies of initial dfs for further manipulation
zip_2013 = init_zip_2013.copy()
zip_2014 = init_zip_2014.copy()
zip_2015 = init_zip_2015.copy()


# Cleaning non-numeric columns
# remove nan's and inf's (turn to 0)

zip_2013.fillna(0, inplace=True)
zip_2013 = zip_2013.replace(np.inf, 0)
zip_2013 = zip_2013[~zip_2013['ZipCode'].isin(['0', 0])]    
zip_2014.fillna(0, inplace=True)
zip_2014 = zip_2014.replace(np.inf, 0)
zip_2014 = zip_2014[~zip_2014['ZipCode'].isin(['0', 0])]
zip_2015.fillna(0, inplace=True)
zip_2015 = zip_2015.replace(np.inf, 0)
zip_2015 = zip_2015[~zip_2015['ZipCode'].isin(['0', 0])]
for i in ['geometry', 'AREA', 'total_gas_incidents']:
    del zip_2013[i]
    del zip_2014[i]
    del zip_2015[i]
   

In [24]:
# processing columns to be in the same order. 
# if 2014 does not have a column from 2013, 
# 0's will be filled for the entire column 

zip_cols_2013 = zip_2013.columns.tolist()
for i in zip_cols_2013:
    if i not in zip_2014.columns:
        zip_2014[i] = 0.0
        
# place 2014 columns in the same order - droppping cols that did not appear in 2013.
zip_2014 = zip_2014[zip_cols_2013]

In [25]:
# validation that zip code orders are the same for our train and test set

for idx, i in enumerate(zip_2013.iloc[:,0].values):
    if zip_2014.iloc[:,0].values[idx] != i:
        print i

for idx, i in enumerate(zip_2014.iloc[:,0].values):
    if zip_2015.iloc[:,0].values[idx] != i:
        print i

In [26]:
# X_train will be 2013 features, y_train will be 2013 gas_leaks_per_bldg_unit
X_train_zip = zip_2013.iloc[:,1:-1].values
y_train_zip = zip_2013.iloc[:,-1].values


# min/max scalling of feature data
min_max_scaler = MinMaxScaler()
X_train_zip = min_max_scaler.fit_transform(X_train_zip)

# X_test will be 2013 features, y_test will be 2014 gas_leaks_per_bldg_unit
X_test_zip = X_train_zip
y_test_zip = zip_2014.iloc[:,-1].values

### DJC NOTE: Someone should actually try doing some Bayesian inference here -- and instead of fitting the X_pred_zip (i.e. 2014) features with MinMaxScaler, actually use parameters learned from the 2013 training set (i.e. prior distribution) and model the change in distribution learned from the 2014 data when cross-validating....

In [27]:
# create prediction features and dependent variable - zip

X_pred_zip = zip_2014.iloc[:,1:-1].values

min_max_scaler = MinMaxScaler()
X_pred_zip = min_max_scaler.fit_transform(X_pred_zip)

y_pred_zip = zip_2015.iloc[:,-1].values


In [28]:
print X_train_zip.shape, X_test_zip.shape, X_pred_zip.shape

(194, 720) (194, 720) (194, 720)


In [29]:
print y_train_zip.shape, y_test_zip.shape, y_pred_zip.shape

(194,) (194,) (194,)


In [30]:
# naive model 2013 to predict 2014

# y_test_zip - y_train_zip # calculate error

# Tract train, test, predict data

In [2]:
# get 2013, 2014, and 2015 data for zipcode

init_tract_2013 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_tract_2013.csv')
init_tract_2014 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_tract_2014.csv')
init_tract_2015 = pd.read_csv('./raw_data/outputs/pluto_fdny_dob_census_to_tract_2015.csv')

### ^^^ DJC NOTE #1: As before, I've changed the name of the initialized dataframes here and created copies below.

### NOTE #2: While before, zip codes were really the index for the observations (and Nate dropped them from the features appropriately), for the tract-level, they actually serve as categorical features that are a rough approximation of geographic proximity and likely have some influence in terms of predicting gas leaks. Not likely better than spatial autocorrelation, but food for thought in case someone tries including zips as features (you'd have to convert to one-hot vector)...

In [3]:
# copy initial dfs
tract_2013 = init_tract_2013.copy()
tract_2014 = init_tract_2014.copy()
tract_2015 = init_tract_2015.copy()


# Cleaning non-numeric columns
# remove nan's and inf's (turn to 0)

tract_2013.fillna(0, inplace=True)
tract_2013 = tract_2013.replace(np.inf, 0)
tract_2014.fillna(0, inplace=True)
tract_2014 = tract_2014.replace(np.inf, 0)
tract_2015.fillna(0, inplace=True)
tract_2015 = tract_2015.replace(np.inf, 0)
for i in ['NTACode', 'NTAName', 'geometry', 'ZipCode', 'total_gas_incidents', 'GEOID']:
    del tract_2013[i]
    del tract_2014[i]
    del tract_2015[i]

In [4]:
# processing columns to be in the same order. 
# if 2014 does not have a column from 2013, 
# 0's will be filled for the entire column 

tract_cols_2013 = tract_2013.columns.tolist()
for i in tract_cols_2013:
    if i not in tract_2014.columns:
        tract_2014[i] = 0.0
        
# place 2014 columns in the same order - droppping cols that did not appear in 2013.
tract_2014 = tract_2014[tract_cols_2013]

In [5]:
# validation that tract orders are the same for our train, test, and predict sets

for idx, i in enumerate(tract_2013.iloc[:,0].values):
    if tract_2014.iloc[:,0].values[idx] != i:
        print i

for idx, i in enumerate(tract_2014.iloc[:,0].values):
    if tract_2015.iloc[:,0].values[idx] != i:
        print i

In [6]:
# X_train will be 2013 features, y_train will be 2013 gas_leaks_per_bldg_unit
X_train_tract = tract_2013.iloc[:,1:-1].values
y_train_tract = tract_2013.iloc[:,-1].values


# min/max scalling of feature data
min_max_scaler = MinMaxScaler()
X_train_tract = min_max_scaler.fit_transform(X_train_tract)

# X_test will be 2013 features, y_test will be 2014 gas_leaks_per_bldg_unit
X_test_tract = X_train_tract
y_test_tract = tract_2014.iloc[:,-1].values

In [7]:
# create prediction features and dependent variable - tract

X_pred_tract = tract_2014.iloc[:,1:-1].values

# scaling of features 
min_max_scaler = MinMaxScaler()
X_pred_tract = min_max_scaler.fit_transform(X_pred_tract)

y_pred_tract = tract_2015.iloc[:,-1].values

In [8]:
print X_pred_tract.shape, X_train_tract.shape, X_test_tract.shape

(3180, 717) (3180, 717) (3180, 717)


In [9]:
print y_train_tract.shape, y_test_tract.shape, y_pred_tract.shape

(3180,) (3180,) (3180,)


In [10]:
pd.options.display.max_seq_items=999
tract_2013.columns

Index([u'TRACT', u'age', u'bldg_class_A0', u'bldg_class_A1', u'bldg_class_A2',
       u'bldg_class_A3', u'bldg_class_A4', u'bldg_class_A5', u'bldg_class_A6',
       u'bldg_class_A7', u'bldg_class_A8', u'bldg_class_A9', u'bldg_class_B1',
       u'bldg_class_B2', u'bldg_class_B3', u'bldg_class_B9', u'bldg_class_C0',
       u'bldg_class_C1', u'bldg_class_C2', u'bldg_class_C3', u'bldg_class_C4',
       u'bldg_class_C5', u'bldg_class_C6', u'bldg_class_C7', u'bldg_class_C8',
       u'bldg_class_C9', u'bldg_class_D0', u'bldg_class_D1', u'bldg_class_D2',
       u'bldg_class_D3', u'bldg_class_D4', u'bldg_class_D5', u'bldg_class_D6',
       u'bldg_class_D7', u'bldg_class_D8', u'bldg_class_D9', u'bldg_class_E1',
       u'bldg_class_E3', u'bldg_class_E4', u'bldg_class_E7', u'bldg_class_E9',
       u'bldg_class_F1', u'bldg_class_F2', u'bldg_class_F4', u'bldg_class_F5',
       u'bldg_class_F8', u'bldg_class_F9', u'bldg_class_G0', u'bldg_class_G1',
       u'bldg_class_G2', u'bldg_class_G3', u'bldg_cl

In [11]:
excludefromfeatures=[
    'TRACT',
    'NTACode',
    'NTAName',
    'geometry',
    'ZipCode',
    'total_gas_incidents'
]

In [12]:
# split training data into features and outcome (numpy arrays, to feed to sklearn algorithms)
label_train = y_train_tract
pred_train = X_train_tract  #.drop(excludefromfeatures, axis=1).fillna(0).replace(np.inf, 0)

# print pred_train.head()
#pred_train = pred_train.values 

min_max_scaler = preprocessing.MinMaxScaler()
pred_train = min_max_scaler.fit_transform(pred_train)
pred_train


array([[ 0.53800359,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.65302504,  0.        ,  0.        , ...,  0.01764281,
         0.        ,  0.24081527],
       [ 0.50357311,  0.        ,  0.17308897, ...,  0.0465952 ,
         0.        ,  0.27688321],
       ..., 
       [ 0.33865839,  0.        ,  0.14366197, ...,  0.07033207,
         0.        ,  0.08666667],
       [ 0.44652925,  0.        ,  0.20103175, ...,  0.01014663,
         0.        ,  0.03860334],
       [ 0.34673118,  0.        ,  0.06091734, ...,  0.        ,
         0.        ,  0.22811671]])

In [46]:
# train the model with 1000 trees, 4 parallel processes, and 10 min samples to split a node 
num_trees = 2500
rf = RandomForestRegressor(n_estimators=num_trees, n_jobs=8, min_samples_split=10, verbose=1, oob_score = True)
rf.fit(X=pred_train, y=label_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   14.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   34.0s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:  3.2min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=10, min_weight_fraction_leaf=0.0,
           n_estimators=2500, n_jobs=8, oob_score=True, random_state=None,
           verbose=1, warm_start=False)

In [47]:
rf.oob_score_

0.41228787758408303

In [28]:
# generate predictions and add them to 'results'
pred_test = X_test_tract

rf_predictions = rf.predict(pred_test)
rf_predictions_tr = rf.predict(pred_train)
#print rf_predictions
#results['preds'] = rf_predictions

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:    1.9s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:    1.9s finis

In [29]:
print sorted(zip(rf.feature_importances_, tract_2014.drop(['TRACT', 'gas_incidents_per_bldg_unit'], axis=1).columns), reverse=True)[:20]

[(0.24650487805509697, 'unit_area'), (0.17051120221006932, 'bldg_class_Q1'), (0.088293339123112533, 'DOB_permit_FO'), (0.074601660340580828, 'bldg_class_Q0'), (0.068878862522407672, 'DOB_permit_NB'), (0.065215169084104282, 'DOB_permit_EQ'), (0.057924554632004563, 'landuse_09'), (0.042694985199927545, 'DOB_permit_EW'), (0.035589798170141115, 'DOB_permit_PL'), (0.031583505836996664, 'DOB_permit_AL'), (0.014585618536002709, 'DOB_permit_OIL'), (0.012281746229994559, 'office_ratio'), (0.0088148484513469703, 'DOB_violation_E-ELEVATOR'), (0.0085213485111224341, 'age'), (0.0084782776035768406, 'total_units'), (0.0069188968774572274, 'bldg_class_Z8'), (0.0066667094752094671, 'value_per_ft'), (0.005967424946306121, 'bldg_class_Q3'), (0.0057588442988810874, 'res_ratio'), (0.0040736355125403486, 'TOTAL_POPULATION')]


In [30]:
rf.feature_importances_.shape

(717,)

In [31]:
zip(rf_predictions, y_test_tract)

[(0.18573045359336088, 0.0),
 (0.82141497592396284, 0.0),
 (0.0062260298826420828, 0.010645224940799999),
 (0.0056473198683418494, 0.0060564599120800002),
 (0.0076901437074535107, 0.00835901652721),
 (0.017131020779234934, 0.018053230719900001),
 (0.017131020779234934, 0.018053230719900001),
 (0.0055935422443136084, 0.0066569051863100007),
 (0.009296688911047412, 0.010778823622900001),
 (12.481395791698811, 24.6114718615),
 (0.0052952221924908033, 0.0051781809921300007),
 (0.008545522556756599, 0.010746497793100001),
 (0.0063725101526201254, 0.007903015194660001),
 (0.0088133006501517445, 0.0114222619844),
 (0.0091816028831622695, 0.0118090190334),
 (0.0091816028831622695, 0.0118090190334),
 (0.0050697405972034461, 0.0054103685340099999),
 (0.0071362991254668596, 0.0087984006734000003),
 (0.0071362991254668596, 0.0087984006734000003),
 (0.065887144930667801, 0.136808080808),
 (0.013990181624972453, 0.023662006782999997),
 (0.0072455491408737402, 0.0073790656626899999),
 (0.007495119250

In [48]:
# train the model with 1000 trees, 4 parallel processes, and 10 min samples to split a node 
num_trees = 2500
rf_s = RandomForestRegressor(n_estimators=num_trees, max_features="sqrt", n_jobs=8, min_samples_split=10, verbose=1, oob_score = True)
rf_s.fit(X=pred_train, y=label_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    6.3s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    8.6s
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:    8.8s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=10, min_weight_fraction_leaf=0.0,
           n_estimators=2500, n_jobs=8, oob_score=True, random_state=None,
           verbose=1, warm_start=False)

In [57]:
rf_s.oob_score_

0.41420019560246801

In [62]:
mt.regression.mean_absolute_error(y_test_tract, rf_s.oob_prediction_)

0.056108334012497041

In [41]:
rf.predict(pred_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:    1.9s finished


array([ 0.2798509 ,  0.82381317,  0.00623787, ...,  0.00776399,
        0.00326866,  0.01397997])

In [42]:
print sorted(zip(rf.feature_importances_, tract_2014.drop(['TRACT', 'gas_incidents_per_bldg_unit'], axis=1).columns), reverse=True)[:20]

[(0.1279289254817699, 'unit_area'), (0.10551084691264309, 'bldg_class_Q1'), (0.092432746746411915, 'landuse_09'), (0.064211724527690445, 'total_units'), (0.062126109526785822, 'bldg_class_Q0'), (0.056090669755936531, 'DOB_permit_EQ'), (0.050422573716084676, 'DOB_permit_NB'), (0.046005492326703805, 'DOB_permit_FO'), (0.034309757890044458, 'DOB_permit_PL'), (0.03405375635182431, 'TOTAL_HOUSEHOLDS'), (0.030000606564393843, 'res_unit_ratio'), (0.026912767795499205, 'age'), (0.026520371922143075, 'office_ratio'), (0.02328643883208029, 'TOTAL_POPULATION'), (0.017241626407264045, 'DOB_permit_EW'), (0.01690196228810233, 'DOB_violation_E-ELEVATOR'), (0.013507493339971154, 'res_ratio'), (0.013393649860505018, 'DOB_permit_AL'), (0.012529583906727417, 'com_ratio'), (0.0094947951205440334, 'MEDIAN_AGE')]


In [43]:
# train the model with 1000 trees, 4 parallel processes, and 10 min samples to split a node 
num_trees = 2500
rf = RandomForestRegressor(criterion="mae", n_estimators=num_trees, max_features="sqrt", n_jobs=8, min_samples_split=10, verbose=1, oob_score = True)
rf.fit(X=pred_train, y=label_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    7.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   39.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  6.5min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  8.9min
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:  9.1min finished


RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=10, min_weight_fraction_leaf=0.0,
           n_estimators=2500, n_jobs=8, oob_score=True, random_state=None,
           verbose=1, warm_start=False)

In [44]:

print sorted(zip(rf.feature_importances_, tract_2014.drop(['TRACT', 'gas_incidents_per_bldg_unit'], axis=1).columns), reverse=True)[:20]

[(0.083083474612921648, 'unit_area'), (0.039036712255202187, 'office_ratio'), (0.038299204990264339, 'bldg_class_Q1'), (0.035771045217175564, 'DOB_permit_FO'), (0.032898102737161386, 'total_units'), (0.032842697763824424, 'landuse_09'), (0.032689328304871007, 'bldg_class_Q0'), (0.025885867006912526, 'bldg_class_U5'), (0.025072631867471537, 'DOB_permit_EQ'), (0.020419045113261751, 'bldg_class_Z8'), (0.018198243416605427, 'value_per_ft'), (0.018068671622720869, 'DOB_permit_NB'), (0.017560671237565106, 'age'), (0.017435480152489583, 'TOTAL_HOUSEHOLDS'), (0.01612409794225441, 'DOB_violation_E-ELEVATOR'), (0.015147196650985108, 'bldg_class_W1'), (0.014970407788741341, 'DOB_permit_PL'), (0.014957355508188658, 'bldg_class_Q3'), (0.014941291482698432, 'bldg_class_P7'), (0.014515280938584698, 'bldg_class_P8')]


In [45]:
rf.oob_score_

0.43879899854885196